In [1]:
import pandas as pd 
from datetime import datetime

In [2]:
#Jobs file reading
df = pd.read_csv('glassdoor_jobs.csv', index_col = 0) 
#Duplicates delete
df.drop_duplicates(inplace=True)

In [3]:
#Job title cleaning
df.dropna(subset='job title', inplace=True)

In [4]:
#Fix seniority
df['job title'] = df['job title'].apply(lambda x : x.replace('Sr.', 'Senior').replace('Sr', 'Senior') \
    if (x.startswith('Sr.') or x.startswith('Sr')) else x)

In [29]:
#Salary parsing
df.dropna(subset=['salary estimate'], inplace=True)
df = df[df['salary estimate']!= '-1']
df = df[~(df['salary estimate'].str.contains('Per Hour'))]
salary = df['salary estimate'].apply(lambda x : x.split('(')[0] if '(' in x else x.split(':')[1])
salary = salary.apply(lambda x : x.replace('K', '').replace('$', ''))
salary_min = salary.apply(lambda x : x.split('-')[0] if '-' in x else x).astype('int')
salary_max = salary.apply(lambda x : x.split('-')[1] if '-' in x else x).astype('int')
df['salary estimate'] = (salary_min + salary_max)/2

In [31]:
df.dropna(subset=['location'], inplace=True)
#Company name text only
def is_float(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

df['company'] = df['company'].apply(lambda x : str(x)[:-3] if is_float(str(x)[-3:]) else x)

In [ ]:
#State codes df
df = df[(df['location']!= 'United States') & (df['location'] !='Pennsylvania')]
df ['state code'] = df['location'].apply(lambda x : 'Remote' if x.lower()=='remote' else x.split(',')[-1])

In [ ]:
#Age of the company
df['company_age'] = datetime.now().year-df['company_founded']

In [ ]:
#Fix Job decsription 
df['job description'].apply(lambda x : x.replace('\r\n', ''))

#Job description parsing (python, java, scala, SQL, etc.)
def set_score(tech_list):
    for tech in tech_list:
        df[tech] = df['job description'].apply(lambda x : 1 if tech in str(x).lower() else 0)

set_score(['python', 'r', 'java', 'scala', 'r', 'sql', 'spark', 'aws', 'azure', 'airflow',
           'snowflake', 'git'])


In [ ]:
#data_score = df.loc[:, 'python':'git'].agg(sum)
df.to_csv('glassdoor_jobs_cleaned.csv', index=False)